# Logits in Causal Language Models

Task: Ask a language model for the most likely next tokens.

## Setup

We start in the same way as the tokenization notebook:

In [1]:
import torch
from torch import tensor
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

One step in this notebook will ask you to write a function. The most common error when function-ifying notebook code is accidentally using a global variable instead of a value computed in the function. This is a quick and dirty little utility to check for that mistake. (For a more polished version, check out [`localscope`](https://localscope.readthedocs.io/en/latest/README.html).)

In [2]:
def check_global_vars(func, allowed_globals):
    import inspect
    used_globals = set(inspect.getclosurevars(func).globals.keys())
    disallowed_globals = used_globals - set(allowed_globals)
    if len(disallowed_globals) > 0:
        raise AssertionError(f"The function {func.__name__} used unexpected global variables: {list(disallowed_globals)}")

Download and load the model.

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", add_prefix_space=True) # smaller version of GPT-2
# Alternative to add_prefix_space is to use `is_split_into_words=True`
# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)

In [4]:
print(f"The tokenizer has {len(tokenizer.get_vocab())} strings in its vocabulary.")
print(f"The model has {model.num_parameters():,d} parameters.")

The tokenizer has 50257 strings in its vocabulary.
The model has 81,912,576 parameters.


## Task

In the tokenization notebook, we simply used the `generate` method to have the model generate some text. Now we'll do it ourselves.

Consider the following phrase:

In [5]:
phrase = "This weekend I plan to"
# Another one to try later. This was a famous early example of the GPT-2 model:
# phrase = "In a shocking finding, scientists discovered a herd of unicorns living in"

1: Call the `tokenizer` on the phrase to get a `batch` that includes `input_ids`.

In [6]:
batch = tokenizer(phrase, return_tensors='pt')
input_ids = batch['input_ids']

2: Call the `model` on the `input_ids`. Examine the shape of the logits.

In [14]:
with torch.no_grad(): # This PyTorch we don't need it to compute gradients for us.
    model_output = model(input_ids)
print(f"logits shape: {list(model_output.logits.shape)} ")

logits shape: [1, 5, 50257] 


3: Pull out the logits corresponding to the *last* token in the input phrase.

In [18]:
last_token_logits = model_output.logits[1]
assert len(last_token_logits.shape) == 1

IndexError: index 1 is out of bounds for dimension 0 with size 1

 4: Identify the token id and corresponding string of the most likely next token.


In [19]:
most_likely_token_id = out.logits[0, -1]
print(f"Most likely next token: {most_likely_token_id}, which corresponds to {repr(tokenizer.decode(most_likely_token_id))}")

NameError: name 'out' is not defined

5: Use the `topk` method to find the top-10 most likely choices for the next token.

*Note*: This uses Pandas to make a nicely displayed table, and a *list comprehension* to decode the tokens. You don't *need* to understand how this all works, but I highly encourage thinking about what's going on.

In [10]:
most_likely_tokens = last_token_logits.topk(...)
most_likely_tokens_df = pd.DataFrame({
    'tokens': [tokenizer.decode(...) for token_id in ....indices],
    'probabilities': most_likely_tokens.values...(dim=0),
})
# Show the table, in a nice formatted way (see https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#Builtin-Styles)
most_likely_tokens_df.style.hide_index().background_gradient()

tokens,probabilities
go,0.188313
take,0.138116
spend,0.099369
make,0.096061
do,0.091928
be,0.088007
attend,0.081478
visit,0.081293
run,0.069480
have,0.065956


6. Write a function that is given a phrase and a *k* and returns the top *k* most likely next tokens.

Build this function using only code that you've already filled in above. Clean up the code so that it doesn't do or display anything extraneous. Add comments about what each step does.

In [11]:
def predict_next_tokens(...):
    # your code here

check_global_vars(predict_next_tokens, ["torch", "tokenizer", "pd", "model"])

In [12]:
predict_next_tokens("This weekend I plan to", 5).style.hide_index().background_gradient()

tokens,probabilities
go,0.306805
take,0.225022
spend,0.161895
make,0.156505
do,0.149773


In [13]:
predict_next_tokens("To be or not to", 5).style.hide_index().background_gradient()

tokens,probabilities
be,0.926792
have,0.030508
the,0.018525
do,0.013536
",",0.010639


In [14]:
predict_next_tokens("For God so loved the", 5).style.hide_index().background_gradient()

tokens,probabilities
world,0.384820
Lord,0.231493
people,0.160956
earth,0.136998
children,0.085733


## Analysis


Q1: Explain the shape of `model_output.logits`.

Q2: What would be required to generate more than one token? What decisions would you have to make?